# Clean data about individual police officers

* Input: `raw_list_of_texas_officers.csv`
* Output: `list_of_texas_officers.csv`
  
##### Author: Everett Wetchler (everett.wetchler@gmail.com)

## 1. Setup and read data

In [1]:
DTW_PROJECT_KEY = 'tji/auxiliary-datasets'
RAW_DATAFRAME_NAME = 'raw_list_of_texas_officers'
AGENCY_COUNTY_DATAFRAME = 'agencies_and_counties'
CLEANED_FILENAME = 'list_of_texas_officers.csv'

In [2]:
####################################################
# Boilerplate import/setup code for general analysis
# everett.wetchler@gmail.com
####################################################

import datetime as dt
import os
import random

import datadotworld as dw
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

## Jupyter setup
%matplotlib inline

## Graphical setup
# Useful colors to reference
SNS_BLUE, SNS_GREEN, SNS_RED, SNS_PURPLE, SNS_YELLOW, SNS_CYAN = sns.color_palette()
SNS_COLORS = sns.color_palette()
# sns.set_palette(sns.color_palette("cubehelix", 8))
mpl.rcParams.update({
  'font.size': 14,
  'axes.titlesize': 'x-large',
  'axes.labelsize': 'large',
  'xtick.labelsize': 'medium',
  'ytick.labelsize': 'medium',
  'legend.fancybox': True,
  'legend.fontsize': 'medium',
  'legend.frameon': True,
  'legend.framealpha': 0.7,
  'figure.figsize': ['9', '6'],
})

# Watermark extension to print version/system information
# Flags:
# -a [author] -d (date) -t (time) -z (timezone) -r (repo)
# -g (git hash) -w (watermark version) -p [packages] (package info)
%load_ext watermark
%watermark -a "Everett Wetchler" -d -t -z -w -p numpy,pandas,matplotlib,datadotworld

####################################################
# END Boilerplate
####################################################

Everett Wetchler 2018-05-12 22:05:32 CDT

numpy 1.14.3
pandas 0.22.0
matplotlib 2.2.0
datadotworld 1.6.0
watermark 1.5.0


In [3]:
from lib.cleaning_tools import validate_gender_cols
from lib.standardize_police_agency_names import standardize_agency_name

In [4]:
datasets = dw.load_dataset(DTW_PROJECT_KEY, force_update=True)
agency_county = datasets.dataframes[AGENCY_COUNTY_DATAFRAME]
agency_county.head()

,agency,county
0,ANDERSON CO CONST PCT 1,ANDERSON
1,ANDERSON CO CONST PCT 2,ANDERSON
2,ANDERSON CO CONST PCT 3,ANDERSON
3,ANDERSON CO CONST PCT 4,ANDERSON
4,ANDERSON CO DIST ATTY OFFICE,ANDERSON


In [5]:
officer_info = datasets.dataframes[RAW_DATAFRAME_NAME]
officer_info.head()

,first_name,middle,last_name,suffix,current_department,highest_cert,service_time,gender
0,BRADFORD,E,TIPPIT,NaN,MISSOURI CITY POLICE DEPT.,Master Peace Officer,23.92,M
1,ROGER,W,KENDALL,NaN,TEXAS BOARD OF CRIMINAL JUSTICE,Master Peace Officer,25.75,M
2,CHARLES,T,KELLEY,NaN,WILLIAMSON CO. SHERIFF'S OFFICE,Master Peace Officer,23.41,M
3,NAYA,C,POPE,NaN,IRVING POLICE DEPT.,Master Peace Officer,36.91,F
4,LINDA,F,HARTT-GOGGIN,NaN,ANTHONY POLICE DEPT.,Master Peace Officer,32.41,F


## 2. Begin cleaning

### Clean gender column

In [6]:
print("Missing gender for %d officers" % officer_info.gender.isnull().sum())
officer_info.gender.value_counts()

Missing gender for 33 officers


M    68131
F     9541
U       12
Name: gender, dtype: int64

In [7]:
officer_info.gender[officer_info.gender == 'U'] = np.nan
officer_info.gender.value_counts()

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


M    68131
F     9541
Name: gender, dtype: int64

In [8]:
validate_gender_cols(officer_info)

### Standardize agency names

In [9]:
officer_info['current_department'] = officer_info['current_department'].apply(standardize_agency_name)
officer_info.head()

,first_name,middle,last_name,suffix,current_department,highest_cert,service_time,gender
0,BRADFORD,E,TIPPIT,NaN,MISSOURI CITY POLICE DEPT,Master Peace Officer,23.92,M
1,ROGER,W,KENDALL,NaN,TEXAS BOARD OF CRIMINAL JUSTICE,Master Peace Officer,25.75,M
2,CHARLES,T,KELLEY,NaN,WILLIAMSON CO SHERIFFS OFFICE,Master Peace Officer,23.41,M
3,NAYA,C,POPE,NaN,IRVING POLICE DEPT,Master Peace Officer,36.91,F
4,LINDA,F,HARTT-GOGGIN,NaN,ANTHONY POLICE DEPT,Master Peace Officer,32.41,F


### Add county names

In [10]:
agency_to_county = dict(zip(agency_county.agency, agency_county.county))
officer_info['current_department_county'] = officer_info['current_department'].apply(lambda d: agency_to_county.get(d, None))
officer_info['current_department_county'].isnull().value_counts()

False    77675
True        42
Name: current_department_county, dtype: int64

In [11]:
officer_info['current_department'][officer_info['current_department_county'].isnull()].value_counts()

ALABAMA-COUSHATTA TRIBE OF TEXAS POLICE DEPT    18
STAFFORD FIRE MARSHALS OFFICE                    6
HIDALGO CO CONST PCT 5                           5
TENAHA POLICE DEPT                               4
UNIV OF DALLAS POLICE DEPT                       2
RICHLAND POLICE DEPT                             2
50TH JUDICIAL DIST ATTY OFFICE                   1
COKE CO CONST PCT                                1
DUMAS FIRE DEPT                                  1
FRANKLIN CO FIRE MARSHALS OFFICE                 1
HAMILTON CO CONST PCT 3                          1
Name: current_department, dtype: int64

### Uppercase string values

In [12]:
for c in officer_info.columns:
    officer_info[c] = officer_info[c].apply(lambda val: val if not isinstance(val, str) else val.upper())
officer_info.head()

,first_name,middle,last_name,suffix,current_department,highest_cert,service_time,gender,current_department_county
0,BRADFORD,E,TIPPIT,NaN,MISSOURI CITY POLICE DEPT,MASTER PEACE OFFICER,23.92,M,FORT BEND
1,ROGER,W,KENDALL,NaN,TEXAS BOARD OF CRIMINAL JUSTICE,MASTER PEACE OFFICER,25.75,M,WALKER
2,CHARLES,T,KELLEY,NaN,WILLIAMSON CO SHERIFFS OFFICE,MASTER PEACE OFFICER,23.41,M,WILLIAMSON
3,NAYA,C,POPE,NaN,IRVING POLICE DEPT,MASTER PEACE OFFICER,36.91,F,DALLAS
4,LINDA,F,HARTT-GOGGIN,NaN,ANTHONY POLICE DEPT,MASTER PEACE OFFICER,32.41,F,EL PASO


### Add 'full_name' column

In [13]:
officer_info['full_name'] = ''
for col in ['first_name', 'middle', 'last_name', 'suffix']:
    officer_info['full_name'] = officer_info['full_name'] + ' ' + officer_info[col].fillna('')
officer_info['full_name'] = officer_info['full_name'].apply(lambda s: ' '.join(s.strip().split()))
officer_info.sample(5)

,first_name,middle,last_name,suffix,current_department,highest_cert,service_time,gender,current_department_county,full_name
20156,KEVIN,L,HOLLEY,NaN,GREENVILLE POLICE DEPT,MASTER PEACE OFFICER,23.75,M,HUNT,KEVIN L HOLLEY
9631,MARK,J,ELKINS,NaN,RICHARDSON POLICE DEPT,MASTER PEACE OFFICER,28.67,M,DALLAS,MARK J ELKINS
2381,STELLA,D,PERKINS,NaN,WEATHERFORD COLLEGE POLICE DEPT,MASTER PEACE OFFICER,22.17,F,PARKER,STELLA D PERKINS
25795,JOSEPH,NaN,DUARTE,NaN,EL PASO POLICE DEPT,MASTER PEACE OFFICER,21.00,M,EL PASO,JOSEPH DUARTE
33303,STEVEN,J,KELLEHER,NaN,KATY POLICE DEPT,ADVANCED PEACE OFFICER,16.75,M,HARRIS,STEVEN J KELLEHER


### Add an integer column for certification level

In [14]:
officer_info['highest_cert'] = officer_info['highest_cert'].fillna('NONE')
cert_levels = {
    'NONE': 0,
    'BASIC PEACE OFFICER': 1,
    'INTERMEDIATE PEACE OFFICER': 2,
    'ADVANCED PEACE OFFICER': 3,
    'MASTER PEACE OFFICER': 4,
}
officer_info['highest_cert_int'] = officer_info['highest_cert'].apply(lambda c: cert_levels[c])

### Reorder and rename columns, sort values

In [15]:
before = officer_info.shape

# Re-order columns
officer_info = officer_info[[
    'first_name', 'middle', 'last_name', 'suffix', 'full_name',
    'gender', 'current_department', 'current_department_county',
    'highest_cert', 'highest_cert_int', 'service_time']]

# Rename columns
officer_info.columns = [
    'name_first', 'name_middle', 'name_last', 'name_suffix', 'name_full',
    'gender','current_agency', 'current_agency_county',
    'highest_cert', 'highest_cert_int', 'service_time']

# Make sure we didn't drop any columns by accident
after = officer_info.shape
assert(before == after)

officer_info.sort_values(['current_agency_county', 'current_agency', 'name_full'], inplace=True)
officer_info.sample(10)

,name_first,name_middle,name_last,name_suffix,name_full,gender,current_agency,current_agency_county,highest_cert,highest_cert_int,service_time
64759,ALEXANDER,M,ZABRISKIE,NaN,ALEXANDER M ZABRISKIE,M,DALLAS POLICE DEPT,DALLAS,INTERMEDIATE PEACE OFFICER,2,4.25
10479,TONY,L,WOODY,NaN,TONY L WOODY,M,FORT BEND CO SHERIFFS OFFICE,FORT BEND,MASTER PEACE OFFICER,4,23.67
11370,JOHNNIE,D,SPARKS,NaN,JOHNNIE D SPARKS,M,TARRANT CO SHERIFFS OFFICE,TARRANT,MASTER PEACE OFFICER,4,32.09
19111,JAMES,H,MINTON,NaN,JAMES H MINTON,M,CROWLEY POLICE DEPT,TARRANT,MASTER PEACE OFFICER,4,28.83
3393,RODNEY,J,SNEED,NaN,RODNEY J SNEED,M,HARRIS CO SHERIFFS OFFICE,HARRIS,MASTER PEACE OFFICER,4,24.50
19072,DENNIS,S,MOORE,NaN,DENNIS S MOORE,M,DUMAS POLICE DEPT,MOORE,ADVANCED PEACE OFFICER,3,10.33
70268,JOSH,NaN,BLACKSTONE,NaN,JOSH BLACKSTONE,M,TITUS CO SHERIFFS OFFICE,TITUS,NONE,0,2.25
3,NAYA,C,POPE,NaN,NAYA C POPE,F,IRVING POLICE DEPT,DALLAS,MASTER PEACE OFFICER,4,36.91
62523,DYLAN,A,HARRIS,NaN,DYLAN A HARRIS,M,TIOGA POLICE DEPT,GRAYSON,NONE,0,1.08
60070,KELLY,S,REYNOLDS,NaN,KELLY S REYNOLDS,M,COLLIN CO SHERIFFS OFFICE,COLLIN,BASIC PEACE OFFICER,1,4.50


## 3. Write

In [16]:
with dw.open_remote_file(DTW_PROJECT_KEY, CLEANED_FILENAME) as w:
    print("Writing to data.world:", CLEANED_FILENAME)
    officer_info.to_csv(w, index=False)

Writing to data.world: list_of_texas_officers.csv
